In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_6

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

In [ ]:
from datasets import load_dataset
dataset_name = "BigTMiami/amazon_helpfulness"
dataset = load_dataset(dataset_name)

#print(train_dataset)
#print(eval_dataset)

#print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# Unipelt

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)
#print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
adapter_hub_name = "unipelt_adapter_classification_trained"

# Load without head, we are adding one in the
adapter_name = model.load_adapter("jgrc3/RobertaDAPT_adapters_unipelt", with_head=False)

# Add head for masked language modeling
model.add_classification_head(
    adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"}, overwrite_ok=True
  )

# Set the adapter to be used for training
model.train_adapter(adapter_name)

(…)b/master/dist/v2/index/roberta-base.json:   0%|          | 0.00/540 [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

In [ ]:
train_dataset = dataset["train"]
eval_dataset = dataset["dev"]
#print_gpu_utilization()

In [ ]:
from transformers import TrainingArguments
from adapters import AdapterTrainer

training_args = TrainingArguments(
    output_dir="./adapter_unipelt_classifier_training_output",
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=100,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
from transformers import TrainingArguments, EarlyStoppingCallback
from adapters import AdapterTrainer

training_args = TrainingArguments(
    output_dir="./adapter_unipelt_classifier_training_output",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5, # Paper: this is for Classification, not domain training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # Lyudmila: changed from 3 to 10 -> used for small roberta
    # Lyudmila: changed back to 3 as agreed
    num_train_epochs=10,
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    save_total_limit=2, # Saves latest 2 checkpoints
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    # Lyudmila: Changed to true -> ot seems according to repo and paper that they used early stopping and used best model
    # Lyudmila: Changed to false as agreed
    load_best_model_at_end=True, #Set to false - we want the last trained model like the paper
    # torch_compile=torch_compile,  # Much Faster
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progress
    metric_for_best_model='f1_macro'
)

# EarlyStoppingCallback with patience
early_stopping = EarlyStoppingCallback(early_stopping_patience=3) # from paper
# callbacks=[early_stopping],

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

# if evaluate_on_test:
#     print("Evaluating model on test set")
#     test_evaluation_result = trainer.evaluate(dataset["test"])
#     print("Test set eval: {0}".format(test_evaluation_result))

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
eval_results = trainer.evaluate(dataset["test"])
# trained_eval_loss = eval_results["eval_loss"]
# print(f"Trained Eval Loss: {trained_eval_loss:.2f}")
# eval_f1_macro = eval_results["eval_f1_macro"]
# print(f"Trained eval_f1_macro:{100.0 * eval_f1_macro:.2f} Eval Loss: {eval_loss:.2f}")
print(eval_results)

In [ ]:
training_results = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.322300,0.331952,0.864000,0.611832
2,0.298500,0.322031,0.868600,0.640422
3,0.294600,0.317394,0.868800,0.648605
4,0.308000,0.323047,0.869600,0.627816
5,0.285300,0.309303,0.870600,0.637654


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.322300,0.331952,0.864000,0.611832
2,0.298500,0.322031,0.868600,0.640422
3,0.294600,0.317394,0.868800,0.648605
4,0.308000,0.323047,0.869600,0.627816
5,0.285300,0.309303,0.870600,0.637654
6,0.289000,0.315229,0.871400,0.660454
7,0.290600,0.311697,0.870800,0.662516
8,0.281900,0.319097,0.872200,0.652092
9,0.297500,0.315731,0.871200,0.653859
10,0.302600,0.316360,0.870400,0.654064


In [ ]:
eval_results = trainer.evaluate(dataset["test"])
# eval_loss = eval_results["eval_loss"]
# eval_f1_macro = eval_results["eval_f1_macro"]
# print(f"Trained eval_f1_macro:{100.0 * eval_f1_macro:.2f} Eval Loss: {eval_loss:.2f}")
print(eval_results)

In [ ]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name
)

pytorch_adapter.bin:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jgrc3/unipelt_adapter_classification_trained/commit/65ff4718ab59186df1c1855d13d8a89911883b7e', commit_message='Upload model', commit_description='', oid='65ff4718ab59186df1c1855d13d8a89911883b7e', pr_url=None, pr_revision=None, pr_num=None)

# Pfeiffer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)
#print_gpu_utilization()

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
adapter_hub_name = "pfeiffer_adapter_classification_trained_10epochs"

# Load without head, we are adding one in the
adapter_name = model.load_adapter("jgrc3/RobertaDAPT_adapters_pfeiffer_reDo", with_head=False)

# Add head for masked language modeling
model.add_classification_head(
    adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"}, overwrite_ok=True
  )

# Set the adapter to be used for training
model.train_adapter(adapter_name)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
summary = model.adapter_summary()
print(summary)

In [ ]:
summary

'================================================================================\nName                     Architecture         #Param      %Param  Active   Train\n--------------------------------------------------------------------------------\nreview_adapter_pfeiffer  bottleneck          894,528       0.718       1       1\n--------------------------------------------------------------------------------\nFull model                               124,645,632     100.000               0\n================================================================================'

In [ ]:
from transformers import TrainingArguments, EarlyStoppingCallback
from adapters import AdapterTrainer

training_args = TrainingArguments(
    output_dir="./adapter_pfeifferv2_classifier_training_output",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5, # Paper: this is for Classification, not domain training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # Lyudmila: changed from 3 to 10 -> used for small roberta
    # Lyudmila: changed back to 3 as agreed
    num_train_epochs=10,
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    save_total_limit=2, # Saves latest 2 checkpoints
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    # Lyudmila: Changed to true -> ot seems according to repo and paper that they used early stopping and used best model
    # Lyudmila: Changed to false as agreed
    load_best_model_at_end=True, #Set to false - we want the last trained model like the paper
    # torch_compile=torch_compile,  # Much Faster
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progress
    metric_for_best_model='f1_macro'
)

# EarlyStoppingCallback with patience
early_stopping = EarlyStoppingCallback(early_stopping_patience=3) # from paper
# callbacks=[early_stopping],

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

# if evaluate_on_test:
#     print("Evaluating model on test set")
#     test_evaluation_result = trainer.evaluate(dataset["test"])
#     print("Test set eval: {0}".format(test_evaluation_result))

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
eval_results = trainer.evaluate(dataset["test"])
eval_results

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6572992205619812,
 'eval_accuracy': 0.85288,
 'eval_f1_macro': 0.4624515981363679,
 'eval_runtime': 235.6196,
 'eval_samples_per_second': 106.103,
 'eval_steps_per_second': 6.634}

In [ ]:
training_results = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.354900,0.334888,0.863000,0.579702
2,0.329100,0.328339,0.867200,0.608671
3,0.312200,0.315204,0.870000,0.670129
4,0.316300,0.316805,0.871000,0.653616
5,0.308600,0.311845,0.870600,0.665556
6,0.315000,0.322853,0.868800,0.610397


In [ ]:
eval_results = trainer.evaluate(dataset["test"])
eval_results

{'eval_loss': 0.31040963530540466,
 'eval_accuracy': 0.87052,
 'eval_f1_macro': 0.6646977800415862,
 'eval_runtime': 234.8089,
 'eval_samples_per_second': 106.47,
 'eval_steps_per_second': 6.656,
 'epoch': 6.0}

In [ ]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name
)

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/jgrc3/pfeiffer_adapter_classification_trained_10epochs/commit/db2ac0ed5b6bf6e5803dadcb4b0dfef0d39a5b7f', commit_message='Upload model', commit_description='', oid='db2ac0ed5b6bf6e5803dadcb4b0dfef0d39a5b7f', pr_url=None, pr_revision=None, pr_num=None)

# Unipelt No Pre

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)
#print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'heads.default.3.bias', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
adapter_hub_name = "unipelt_adapter_classification_noPre"
adapter_name = "classifier_unipelt_no_pretraining"
adapter_type = "unipelt" # could be "lora", etc.

# Add a new adapter
model.add_adapter(adapter_name, config=adapter_type)

# Add head for classification modeling
model.add_classification_head(
    adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
model.train_adapter(adapter_name)

In [ ]:
summary = model.adapter_summary()
summary

'================================================================================\nName                     Architecture         #Param      %Param  Active   Train\n--------------------------------------------------------------------------------\nclassifier_unipelt_no_pretrainingunion            11,083,376       8.892       1       1\n--------------------------------------------------------------------------------\nFull model                               124,645,632     100.000               0\n================================================================================'

In [ ]:
from transformers import TrainingArguments, EarlyStoppingCallback
from adapters import AdapterTrainer

training_args = TrainingArguments(
    output_dir="./adapter_unipeltNone_classifier_training_output",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5, # Paper: this is for Classification, not domain training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # Lyudmila: changed from 3 to 10 -> used for small roberta
    # Lyudmila: changed back to 3 as agreed
    num_train_epochs=10,
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    save_total_limit=2, # Saves latest 2 checkpoints
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    # Lyudmila: Changed to true -> ot seems according to repo and paper that they used early stopping and used best model
    # Lyudmila: Changed to false as agreed
    load_best_model_at_end=True, #Set to false - we want the last trained model like the paper
    # torch_compile=torch_compile,  # Much Faster
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progress
    metric_for_best_model='f1_macro'
)

# EarlyStoppingCallback with patience
early_stopping = EarlyStoppingCallback(early_stopping_patience=3) # from paper
# callbacks=[early_stopping],

uni_no_trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
eval_results = uni_no_trainer.evaluate(dataset["test"])
eval_results

{'eval_loss': 0.717073917388916,
 'eval_accuracy': 0.1466,
 'eval_f1_macro': 0.12785627071341357,
 'eval_runtime': 263.5958,
 'eval_samples_per_second': 94.842,
 'eval_steps_per_second': 5.93}

In [ ]:
training_results = uni_no_trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.338300,0.325178,0.863000,0.641847
2,0.341400,0.326409,0.865400,0.577679
3,0.339300,0.318077,0.866800,0.634540
4,0.321900,0.313238,0.868800,0.657291
5,0.296700,0.317541,0.867400,0.634587


In [ ]:
eval_results = uni_no_trainer.evaluate(dataset["test"])
eval_results

{'eval_loss': 0.3080004155635834,
 'eval_accuracy': 0.8724,
 'eval_f1_macro': 0.6604338299092234,
 'eval_runtime': 261.014,
 'eval_samples_per_second': 95.78,
 'eval_steps_per_second': 5.988,
 'epoch': 5.0}

In [ ]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name
)

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/jgrc3/unipelt_adapter_classification_noPre/commit/d4b793b11521dd7f863a0b4ee8d9082fe8cfb04e', commit_message='Upload model', commit_description='', oid='d4b793b11521dd7f863a0b4ee8d9082fe8cfb04e', pr_url=None, pr_revision=None, pr_num=None)

# Pfeiffer No Pre Redo

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)
#print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
adapter_hub_name = "pfeiffer_adapter_classification_noPre_10epochs_redo"
adapter_name = "classifier_pfeiffer_no_pretraining"
adapter_type = "pfeiffer" # could be "lora", etc.

# Add a new adapter
model.add_adapter(adapter_name, config=adapter_type)

# Add head for classification modeling
model.add_classification_head(
    adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
model.train_adapter(adapter_name)

In [ ]:
summary = model.adapter_summary()
summary

'================================================================================\nName                     Architecture         #Param      %Param  Active   Train\n--------------------------------------------------------------------------------\nclassifier_pfeiffer_no_pretrainingbottleneck          894,528       0.718       1       1\n--------------------------------------------------------------------------------\nFull model                               124,645,632     100.000               0\n================================================================================'

In [ ]:
from transformers import TrainingArguments, EarlyStoppingCallback
from adapters import AdapterTrainer

training_args = TrainingArguments(
    output_dir="./adapter_pfeifferNoPre_redo_classifier_training_output",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5, # Paper: this is for Classification, not domain training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # Lyudmila: changed from 3 to 10 -> used for small roberta
    # Lyudmila: changed back to 3 as agreed
    num_train_epochs=10,
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    save_total_limit=2, # Saves latest 2 checkpoints
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    # Lyudmila: Changed to true -> ot seems according to repo and paper that they used early stopping and used best model
    # Lyudmila: Changed to false as agreed
    load_best_model_at_end=True, #Set to false - we want the last trained model like the paper
    # torch_compile=torch_compile,  # Much Faster
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progress
    metric_for_best_model='f1_macro'
)

# EarlyStoppingCallback with patience
early_stopping = EarlyStoppingCallback(early_stopping_patience=3) # from paper
# callbacks=[early_stopping],

pfeiffer_no_trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
eval_results = pfeiffer_no_trainer.evaluate(dataset["test"])
eval_results

{'eval_loss': 0.7857111692428589,
 'eval_accuracy': 0.1466,
 'eval_f1_macro': 0.12785627071341357,
 'eval_runtime': 234.2563,
 'eval_samples_per_second': 106.721,
 'eval_steps_per_second': 6.672}

In [ ]:
training_results = pfeiffer_no_trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.339600,0.325591,0.864800,0.660681
2,0.347000,0.323949,0.868000,0.598676
3,0.334300,0.317878,0.870000,0.627586
4,0.328300,0.315940,0.870400,0.641908


In [ ]:
eval_results = pfeiffer_no_trainer.evaluate(dataset["test"])
eval_results

{'eval_loss': 0.3205760717391968,
 'eval_accuracy': 0.86376,
 'eval_f1_macro': 0.6514272431268903,
 'eval_runtime': 232.2539,
 'eval_samples_per_second': 107.641,
 'eval_steps_per_second': 6.73,
 'epoch': 4.0}

In [ ]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jgrc3/pfeiffer_adapter_classification_noPre_10epochs_redo/commit/84abb6d5864f98bc1d01235bd928f4ed7050ae96', commit_message='Upload model', commit_description='', oid='84abb6d5864f98bc1d01235bd928f4ed7050ae96', pr_url=None, pr_revision=None, pr_num=None)

# Updated Hyperparams

# Unipelt LR = 0.0001

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
adapter_hub_name = "unipelt_adapter_classification_trained_lr0_0001"

# Load without head, we are adding one in the
adapter_name = model.load_adapter("jgrc3/RobertaDAPT_adapters_unipelt", with_head=False)

# Add head for masked language modeling
model.add_classification_head(
    adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"}, overwrite_ok=True
  )

# Set the adapter to be used for training
model.train_adapter(adapter_name)

(…)b/master/dist/v2/index/roberta-base.json:   0%|          | 0.00/540 [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

In [ ]:
summary = model.adapter_summary()
summary

'================================================================================\nName                     Architecture         #Param      %Param  Active   Train\n--------------------------------------------------------------------------------\nreview_adapter_unipelt   union            11,083,376       8.892       1       1\n--------------------------------------------------------------------------------\nFull model                               124,645,632     100.000               0\n================================================================================'

In [ ]:
from transformers import TrainingArguments, EarlyStoppingCallback
from adapters import AdapterTrainer

training_args = TrainingArguments(
    output_dir="./adapter_unipelt_classifier_training_output",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4, # Paper: this is for Classification, not domain training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # Lyudmila: changed from 3 to 10 -> used for small roberta
    # Lyudmila: changed back to 3 as agreed
    num_train_epochs=10,
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    save_total_limit=2, # Saves latest 2 checkpoints
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    # Lyudmila: Changed to true -> ot seems according to repo and paper that they used early stopping and used best model
    # Lyudmila: Changed to false as agreed
    load_best_model_at_end=True, #Set to false - we want the last trained model like the paper
    # torch_compile=torch_compile,  # Much Faster
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progress
    metric_for_best_model='f1_macro'
)

# EarlyStoppingCallback with patience
early_stopping = EarlyStoppingCallback(early_stopping_patience=3) # from paper
# callbacks=[early_stopping],

uni_pre_trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
eval_results = uni_pre_trainer.evaluate(dataset["test"])
eval_results

KeyboardInterrupt: 

In [ ]:
training_results = uni_pre_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.312300,0.329392,0.865600,0.597778
2,0.284000,0.309012,0.871800,0.668658
3,0.285400,0.306885,0.873200,0.674644
4,0.300700,0.309931,0.875400,0.661974
5,0.263600,0.311107,0.874600,0.665557
6,0.258200,0.320848,0.874800,0.693386
7,0.255900,0.321044,0.876000,0.692611
8,0.239400,0.351591,0.873200,0.652100
9,0.235700,0.337358,0.872400,0.691224


In [ ]:
eval_results = uni_pre_trainer.evaluate(dataset["test"])
eval_results

{'eval_loss': 0.31532949209213257,
 'eval_accuracy': 0.87644,
 'eval_f1_macro': 0.6921446774828509,
 'eval_runtime': 262.2671,
 'eval_samples_per_second': 95.323,
 'eval_steps_per_second': 5.96,
 'epoch': 9.0}

In [ ]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jgrc3/unipelt_adapter_classification_trained_lr0_0001/commit/1e50ecd4c11440a380a2f5f2e2fff727a78e3574', commit_message='Upload model', commit_description='', oid='1e50ecd4c11440a380a2f5f2e2fff727a78e3574', pr_url=None, pr_revision=None, pr_num=None)